In [ ]:
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir('./drive/MyDrive/nixtlats')
# print(os.getcwd())

In [ ]:
# import os
# os.chdir('..')
# print(os.getcwd())

/home/ubuntu/nixtlats/nbs


In [ ]:
# !pip install pytorch-lightning
# !pip install torchinfo
# !pip install fastcore
# !pip install s3fs
# !pip install patool
# !pip install --upgrade pandas==1.2.4
# !pip install --upgrade requests==2.25.1
# !pip install --upgrade scikit-learn==0.24

In [ ]:
#default_exp models.nbeats.tuning

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from dataclasses import dataclass, field
from itertools import product
from pathlib import Path
import shutil
from typing import Callable, Dict, Iterable, Union, List
from tqdm import tqdm
import pylab as plt
from pylab import rcParams
import math
from IPython.display import clear_output
import torch
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping
import time
import pickle
import argparse

from nixtlats.models.nbeats.nbeats import NBEATS
from nixtlats.data.datasets.tourism import TourismInfo, Tourism
from nixtlats.data.datasets.m4 import M4Info, M4, M4Evaluation
from nixtlats.data.tsdataset import WindowsDataset
from nixtlats.data.tsloader import TimeSeriesLoader
from nixtlats.experiments.utils import create_datasets, get_mask_dfs

# NBEATS Hyperparameters

In [ ]:
#export
@dataclass
class NBEATSHyperparameters:

    dataset: str
    frequency: str
    group: type = field(init=False)
    freq_grid: dict = field(init=False)
    ensemble_grid: dict
    grid: dict = field(init=False)

    # Common grid to all frequencies
    common_grid = {}

    # Architecture parameters
    common_grid['activation'] = ['ReLU'] # Oreshkin
    common_grid['n_x'] = [0] # No exogenous variables
    common_grid['n_s'] = [0] # No static variables
    common_grid['n_x_hidden'] = [0] # No exogenous variables
    common_grid['n_s_hidden'] = [0] # No static variables
    common_grid['stack_types'] = [['trend', 'seasonality']] # NBEATS-I original architecture
    common_grid['n_blocks'] = [[3, 3]] # Trend blocks, Seasonal blocks - Oreshkin
    common_grid['n_layers'] = [[4, 4]] # Trend-block layers, Seasonal-block - Oreshkin
    common_grid['shared_weights'] = [True] # Oreshkin
    common_grid['n_harmonics'] = [1] # Oreshkin
    common_grid['n_polynomials'] = [2] # Trend polynomial degree
    common_grid['n_theta_hidden'] = [[common_grid['n_layers'][0][0] * [256],
                                    common_grid['n_layers'][0][1] * [2048]]] # Oreshkin
    common_grid['initialization'] = ['lecun_normal'] # Arbitrary

    # Optimization parameters
    # common_grid['learning_rate'] = [0.0001] # Oreshkin
    common_grid['lr_decay'] = [0] # No lr_decay in the original implementation
    common_grid['lr_decay_step_size'] = [1_000] # No lr_decay in the original implementation
    common_grid['dropout_prob_theta'] = [0] # No dropout in the original implementation
    common_grid['weight_decay'] = [0] # # No weight_decay in the original implementation
    common_grid['batch_size'] = [1024] # Oreshkin
    common_grid['batch_normalization'] = [False] # No batch_normalization in the original implementation
    common_grid['train_sample_freq'] = [1] 
    
    def __post_init__(self):
        
        
        assert self.dataset in ['M4', 'Tourism'], \
            f"Dataset must be one of the following: {['M4', 'Tourism']}"
        
        if self.dataset == 'M4': 
            assert self.frequency in ['Yearly', 'Quarterly', 'Monthly', 'Weekly', 'Daily', 'Hourly'], \
                f"Frequency must be one of the following: {['Yearly', 'Quarterly', 'Monthly', 'Weekly', 'Daily', 'Hourly']}"
            self.group = M4Info[self.frequency]
            self.common_grid['l_h'] = [10 if self.frequency in ['Weekly', 'Daily', 'Hourly'] else 1.5]
            self.common_grid['loss_val'] = ['SMAPE']
        if self.dataset == 'Tourism': 
            assert self.frequency in ['Yearly', 'Quarterly', 'Monthly'], \
                f"Frequency must be one of the following: {['Yearly', 'Quarterly', 'Monthly']}"
            self.group = TourismInfo[self.frequency]
            self.common_grid['l_h'] = [10 if self.frequency in ['Quarterly'] else 20]
            self.common_grid['loss_val'] = ['MAPE']
        
        # Frequency grid
        self.freq_grid = {}
        self.freq_grid['n_time_in'] = [self.group.horizon * i for i in self.ensemble_grid['lookbacks']]
        self.freq_grid['n_time_out'] = [self.group.horizon]
        self.freq_grid['frequency'] = [self.group.freq]
        self.freq_grid['seasonality'] = [self.group.seasonality]

        # Ensemble grid
        self.ensemble_grid = self.ensemble_grid.copy()
        self.ensemble_grid.pop('lookbacks')

        # Full hyperparameter grid
        self.grid = {**self.common_grid,
                     **self.freq_grid}

# Loaders

In [ ]:
#export
def complete_series(Y_df, hparams):
    Y_complete_df = []
    series_counts = Y_df.iloc[:,:2].groupby(['unique_id']).agg('count').reset_index()
    series_counts.rename(columns={series_counts.columns[1]: 'n'}, inplace=True)
    n_in_out = hparams['n_time_in'] + hparams['n_time_out']

    for serie_id in tqdm(Y_df.unique_id.unique(), position=0, leave=True):
        serie_count = series_counts[series_counts.unique_id == serie_id].n.iloc[0]
        Y_df_id = Y_df[Y_df.unique_id == serie_id]
        if serie_count < n_in_out:
            reps = n_in_out - serie_count
            Y_complete_df.append(pd.concat([Y_df_id.iloc[[0],:]] * reps + [Y_df_id], axis=0))
        else:
            Y_complete_df.append(Y_df_id)

    Y_complete_df = pd.concat(Y_complete_df, axis=0).reset_index(drop=True)

    return Y_complete_df

In [ ]:
#export
def create_loaders(Y_df, S_df, dataset, hparams, num_workers):

    print(f'Instantiating loaders (n_time_in = {hparams["n_time_in"]})...')

    # To prevent series shorter than n_time_in + n_time_out
    if dataset == 'Tourism': series_df = complete_series(Y_df, hparams)
    else: series_df = Y_df
    
    train_mask_df, val_mask_df, _ = get_mask_dfs(Y_df=series_df,
                                                        ds_in_test=0,
                                                        ds_in_val=hparams['n_time_out'])

    train_dataset = WindowsDataset(Y_df=series_df, S_df=S_df,
                                   mask_df=train_mask_df,
                                   input_size=hparams['n_time_in'],
                                   output_size=hparams['n_time_out'],
                                   sample_freq=hparams['train_sample_freq'],
                                   complete_sample=False)

    test_dataset = WindowsDataset(Y_df=series_df, S_df=S_df,
                                    mask_df=val_mask_df,
                                    input_size=hparams['n_time_in'],
                                    output_size=hparams['n_time_out'],
                                    sample_freq=hparams['train_sample_freq'],
                                    complete_sample=True)
    
    train_loader = TimeSeriesLoader(dataset=train_dataset,
                                    batch_size=int(hparams['batch_size']),
                                    eq_batch_size=True,
                                    num_workers=4,
                                    shuffle=True)
    
    val_loader = TimeSeriesLoader(dataset=train_dataset,
                                    batch_size=int(hparams['batch_size']),
                                    eq_batch_size=True,
                                    num_workers=4,
                                    shuffle=False)
        
    test_loader = TimeSeriesLoader(dataset=test_dataset,
                                        batch_size=int(hparams['batch_size']),
                                        eq_batch_size=False,
                                        num_workers=4,
                                        shuffle=False)
    
    print('\nData loaders ready.\n')
    
    del train_dataset, test_dataset
    
    return train_loader, val_loader, test_loader

# Utils

In [ ]:
#export
def _parameter_grid(grid):
    specs_list = list(product(*list(grid.values())))
    model_specs_df = pd.DataFrame(specs_list, columns=list(grid.keys()))
    
    return model_specs_df

In [ ]:
#export
def print_models_list(dataset: str, frequency: str, ensemble_grid: dict, table_width: int):
    freq = NBEATSHyperparameters(dataset=dataset, frequency=frequency, ensemble_grid=ensemble_grid)

    freq_grid_table = pd.Series({**freq.grid, **freq.ensemble_grid})
    freq_table_header  = f'\n{freq.group.name} '
    freq_table_header += 'grid (# of different model configurations = '
    freq_table_header += f'{len(_parameter_grid({**freq.grid, **freq.ensemble_grid}))}):\n'      
    print(f'{freq_table_header}{table_width*"="}\n{freq_grid_table}\n{table_width*"="}\n')

In [ ]:
#export
def show_tensorboard(logs_path, model_path):
    logs_model_path = f'{logs_path}/{model_path}'
#     %load_ext tensorboard
#     %tensorboard --logdir $logs_model_path
    # os.system('load_ext tensorboard')
    # os.system('tensorboard --logdir $logs_model_path')

In [ ]:
#export
def tourism_evaluation(y_hat_list, models_list, Y_df, S_df, dataset, hparams, num_workers):
    _, _, test_loader = create_loaders(Y_df=Y_df, 
                                       S_df=S_df, 
                                       dataset=dataset,
                                       hparams=hparams, 
                                       num_workers=num_workers)

    mape_list = []

    for y_hat in tqdm(y_hat_list, position=0, leave=True):

        y_full_batches = np.vstack([batch['Y'].detach().numpy() for batch in test_loader])
        sample_mask_full_batches = np.vstack([batch['sample_mask'].detach().numpy() for batch in test_loader])
        y_true = y_full_batches * sample_mask_full_batches
        y_true = pd.DataFrame(np.reshape(y_true[sample_mask_full_batches ==  1], 
                                            (y_full_batches.shape[0],-1))).to_numpy()
        mape_list.append(100 * np.nanmean(np.nanmean(np.abs(y_true - y_hat) / np.abs(y_true), axis=1), axis=0))

    if len(y_hat_list) == 1:
        return mape_list[0]
    else:
        return pd.DataFrame.from_dict({'model': models_list, 'MAPE': mape_list})

In [ ]:
#export
def m4_evaluation(y_hat_list, models_list, frequency):

    results_list = []

    for i, y_hat in enumerate(tqdm(y_hat_list, position=0, leave=True)):

        results = {}
        
        m4_evaluation = M4Evaluation.evaluate('data', frequency, y_hat)
        results['model'] = models_list[i]
        results['SMAPE'] = m4_evaluation['SMAPE'].item()
        results['MASE'] = m4_evaluation['MASE'].item()
        results['OWA'] = m4_evaluation['OWA'].item()

        results_list.append(results)

    if len(y_hat_list) == 1:
        return results_list[0]
    else:
        return pd.DataFrame.from_dict(results_list)

# NBEATS Ensemble

In [ ]:
#export
def nbeats_instantiate(hparams):

    model = NBEATS(n_time_in=int(hparams['n_time_in']),
                   n_time_out=int(hparams['n_time_out']),
                   n_x=hparams['n_x'],
                   n_s=hparams['n_s'],
                   n_s_hidden=int(hparams['n_s_hidden']),
                   n_x_hidden=int(hparams['n_x_hidden']),
                   shared_weights=hparams['shared_weights'],
                   initialization=hparams['initialization'],
                   activation=hparams['activation'],
                   stack_types=hparams['stack_types'],
                   n_blocks=hparams['n_blocks'],
                   n_layers=hparams['n_layers'],
                   n_theta_hidden=hparams['n_theta_hidden'],
                   n_harmonics=int(hparams['n_harmonics']),
                   n_polynomials=int(hparams['n_polynomials']),
                   batch_normalization = hparams['batch_normalization'],
                   dropout_prob_theta=hparams['dropout_prob_theta'],
                   learning_rate=float(hparams['learning_rate']),
                   lr_decay=float(hparams['lr_decay']),
                   lr_decay_step_size=float(hparams['lr_decay_step_size']),
                   weight_decay=hparams['weight_decay'],
                   loss_train=hparams['loss_train'],
                   loss_hypar=int(hparams['seasonality']),
                   loss_valid=hparams['loss_val'],
                   frequency=hparams['frequency'],
                   seasonality=int(hparams['seasonality']),
                   random_seed=int(hparams['random_seed']))
    
    return model

In [ ]:
#export
class NBEATSEnsemble:

    def __init__(self, 
                 dataset: str,
                 frequency: str, 
                 ensemble_grid: dict,
                 use_gpus: bool=False, gpus: int=None, auto_select_gpus: bool=False):
        
        if use_gpus:
            assert isinstance(gpus, (int, list, str)), \
                f'if use_gpus == True, gpus must be {int}, {list} or {str}, not {type(gpus)}.'
            if (isinstance(gpus, int)):
                assert gpus > 0 or gpus == -1, \
                    f'if gpus is of type {int}, it must be either a positive integer or equal to -1.'
        else:
            assert gpus == None, f'if use_gpus == False, gpus must be {None}, not {type(gpus)}.'

        self.dataset = dataset
        self.frequency = NBEATSHyperparameters(dataset=dataset,
                                               frequency=frequency, 
                                               ensemble_grid=ensemble_grid)
        self.gpus = gpus
        self.auto_select_gpus = auto_select_gpus

    def fit(self,
            loader: callable,
            val_freq_steps: int,
            early_stopping_params: dict,
            tensorboard_logs: bool,
            logs_path: str,
            save_file_path: str,
            num_workers: int):

        idx_ensemble = 0

        if self.dataset == 'M4': Y_df, _, S_df = M4.load(directory='data', group=self.frequency.group.name)
        if self.dataset == 'Tourism': Y_df, _, S_df = Tourism.load(directory='data', group=self.frequency.group.name)

        freq_grid = _parameter_grid(self.frequency.grid)
        forecasts = []

        if tensorboard_logs and Path(f'{logs_path}/{self.frequency.group.name}').exists():
            shutil.rmtree(f'{logs_path}/{self.frequency.group.name}')
            show_tensorboard(logs_path=logs_path, model_path=self.frequency.group.name)

        for idx_hparams, row_hparams in freq_grid.iterrows():                             
            hparams = row_hparams.to_dict()
            train_loader, val_loader, test_loader = create_loaders(Y_df=Y_df, 
                                                                   S_df=S_df, 
                                                                   dataset=self.dataset,
                                                                   hparams=hparams, 
                                                                   num_workers=num_workers)

            ensemble_grid = _parameter_grid(self.frequency.ensemble_grid)

            for idx_ensemble_hparams, row_ensemble_hparams in ensemble_grid.iterrows():
                clear_output(wait=True)
                idx_ensemble += 1
                hparams_ensemble = {**hparams, **row_ensemble_hparams.to_dict()}

                model = nbeats_instantiate(hparams_ensemble)
                self.print_model_version(hparams_ensemble, idx_ensemble)

                if tensorboard_logs: logger = self.create_logger(hparams_ensemble,
                                                                 logs_path)
                else: logger = False

                
                early_stop = EarlyStopping(monitor='val_loss',
                                        min_delta=early_stopping_params['min_delta'],
                                        patience=early_stopping_params['patience'],
                                        strict=early_stopping_params['strict'],
                                        verbose=False,
                                        mode='min',
                                        check_on_train_epoch_end=False)

                trainer = pl.Trainer(max_steps=hparams_ensemble['n_steps'],
                                     gradient_clip_val=0,
                                     progress_bar_refresh_rate=50, 
                                     gpus=self.gpus,
                                     auto_select_gpus=self.auto_select_gpus, 
                                     check_val_every_n_epoch=val_freq_steps,
                                     checkpoint_callback=tensorboard_logs,
                                     callbacks=[early_stop],
                                     logger=logger)           
                  
                # trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)
                trainer.fit(model, train_dataloader=train_loader, val_dataloaders=test_loader)
                
                if self.gpus != 1:
                    #distributed training
                    #predict not supported, only using 1 gpu
                    trainer = pl.Trainer(max_steps=hparams_ensemble['n_steps'],
                                     gradient_clip_val=0,
                                     progress_bar_refresh_rate=50, 
                                     gpus=1 if torch.cuda.is_available() else None,
                                     auto_select_gpus=self.auto_select_gpus, 
                                     check_val_every_n_epoch=val_freq_steps,
                                     checkpoint_callback=tensorboard_logs,
                                     logger=logger)    
                    
                outputs = trainer.predict(model, test_loader)
                outputs_df = self.outputs_to_df(outputs, hparams_ensemble)
                forecasts.append(outputs_df.copy())

                del trainer, model, outputs, outputs_df

            del train_loader, val_loader, test_loader

        ensembled_forecasts = pd.concat(forecasts).groupby('unique_id').median(0)
        ensembled_forecasts.reset_index(inplace=True) 
        ensembled_forecasts.drop(columns='unique_id', inplace=True)

        # Results
        time_id = str(time.time()).split(sep='.')[-1]

        hparams = freq_grid.iloc[0].to_dict()

        results_file_name  = save_file_path
        results_file_name += f'/{self.dataset}_{self.frequency.group.name}_{time_id}.pkl'
        results = {'dataset': self.frequency.dataset,
                   'frequency': self.frequency.frequency,
                   'Y_df': Y_df,
                   'S_df': S_df,
                   'hparams': hparams,
                   'forecasts_list': forecasts,
                   'ensembled_forecasts': ensembled_forecasts,
                   'file_name': results_file_name}

        
        if self.dataset == 'Tourism':
            results['MAPE'] = tourism_evaluation(y_hat_list=[ensembled_forecasts.to_numpy()], 
                                                 models_list=['ensemble'],
                                                 Y_df=Y_df, 
                                                 S_df=S_df, 
                                                 dataset=self.dataset, 
                                                 hparams=hparams, 
                                                 num_workers=num_workers)

        if self.dataset == 'M4':            
            m4_results = m4_evaluation(y_hat_list=[ensembled_forecasts.to_numpy()],
                                       models_list=['ensemble'],
                                       frequency=self.frequency.frequency)

            results['SMAPE'] = m4_results['SMAPE']
            results['MASE'] = m4_results['MASE']
            results['OWA'] = m4_results['OWA']
        
        with open(results_file_name, 'wb') as f: pickle.dump(results, f)        

        return results

    def outputs_to_df(self, outputs, hparams):

        version  = f'loss-{hparams["loss_train"]}_'
        version += f'lbl-{hparams["n_time_in"] // self.frequency.group.horizon}_'
        version += f'rs-{hparams["random_seed"]}_'
        version += f'lr-{hparams["learning_rate"]}'

        outputs_df = torch.vstack([outputs[i][1] \
                                   for i in range(len(outputs))]).detach().cpu().numpy()
        outputs_df = pd.DataFrame(outputs_df)
        outputs_df.insert(0, 'unique_id', np.arange(outputs_df.shape[0]))
        outputs_df.insert(1, 'model', version)

        return outputs_df

    def create_logger(self, hparams, logs_path):
        name = self.frequency.group.name
        version  = f'loss-{hparams["loss_train"]}_'
        version += f'lbl-{hparams["n_time_in"] // self.frequency.group.horizon}_'
        version += f'rs-{hparams["random_seed"]}_'
        version += f'lr-{hparams["learning_rate"]}'

        logger = TensorBoardLogger(logs_path, name=name, version=version, default_hp_metric=False)

        return logger

    def print_model_version(self, hparams, idx_ensemble):
        n_models = len(self.frequency.ensemble_grid['loss_train']) * \
                   len(self.frequency.grid['n_time_in']) * \
                   len(self.frequency.ensemble_grid['random_seed']) * \
                   len(self.frequency.ensemble_grid['learning_rate'])
        model_version  = f'\n{self.frequency.group.name} ({idx_ensemble}/{n_models}) - '
        model_version += f'loss: {hparams["loss_train"]}, ' 
        model_version += f'lookback length: {hparams["n_time_in"] // self.frequency.group.horizon}, '
        model_version += f'random_seed: {hparams["random_seed"]} '
        model_version += f'learning_rate: {hparams["learning_rate"]}'
        print(model_version)

# Ensemble Experiments

In [ ]:
#export
def main(args):
    
    FREQUENCY = args.frequency
    DATASET = args.dataset
    LOGS_PATH = Path(f'lightning_logs/{DATASET}')
    RESULTS_FILE_PATH = 'nbs/nbeats_results'
    NUM_WORKERS = 4
    tensorboard_logs = False

    ensemble_grid = {'loss_train': ['SMAPE'],
#                      'loss_train': ['MAPE', 'SMAPE', 'MASE'],
                     'lookbacks': list(range(2, 8)),
                     'random_seed': np.arange(args.n_random_seeds).tolist(),
                     'learning_rate': np.exp(np.random.uniform(np.log(1e-2), np.log(1e-5), args.n_lr)).tolist(),
                     'n_steps': [args.n_iterations]}

    print_models_list(dataset=DATASET, frequency=FREQUENCY, ensemble_grid=ensemble_grid, table_width=72)
    
    ensemble = NBEATSEnsemble(dataset=DATASET,
                          frequency=FREQUENCY,
                          ensemble_grid=ensemble_grid,
                          use_gpus=True if torch.cuda.is_available() else False, 
                          gpus=-1 if torch.cuda.is_available() else None, 
                          auto_select_gpus=True if torch.cuda.is_available() else False)

    start = time.time()
    val_freq_steps = 115

    early_stopping_params = {'min_delta': 0.0, 'patience': args.es_patience, 'strict': False}

    forecasts = ensemble.fit(loader=TimeSeriesLoader,  
                             val_freq_steps=val_freq_steps,
                             early_stopping_params=early_stopping_params,
                             tensorboard_logs=tensorboard_logs,
                             logs_path=LOGS_PATH,   
                             save_file_path=RESULTS_FILE_PATH,
                             num_workers=NUM_WORKERS)

    print(f"Results in file {forecasts['file_name']}")
    print(f'Elapsed time: {time.time() - start}')
    
    import pickle
    with open(forecasts['file_name'], 'rb') as f: ensemble_results = pickle.load(f)

    forecasts_by_model = ensemble_results['forecasts_list']

    if ensemble_results['dataset'] == 'Tourism':
        model_performances = tourism_evaluation(y_hat_list=[fcst.iloc[:, 2:].to_numpy() for fcst in forecasts_by_model], 
                                                models_list=[fcst.model[0] for fcst in forecasts_by_model],
                                                Y_df=ensemble_results['Y_df'], 
                                                S_df=ensemble_results['S_df'], 
                                                dataset=ensemble_results['dataset'], 
                                                hparams=ensemble_results['hparams'], 
                                                num_workers=NUM_WORKERS)

    if ensemble_results['dataset'] == 'M4':
        model_performances = m4_evaluation(y_hat_list=[fcst.iloc[:, 2:].to_numpy() for fcst in forecasts_by_model],
                                           models_list=[fcst.model[0] for fcst in forecasts_by_model],
                                           frequency=FREQUENCY)
        
    print(model_performances)

In [ ]:
# export
def parse_args():
    desc = "NBEATS LR and Random Seed tuning"
    parser = argparse.ArgumentParser(description=desc)
    parser.add_argument('--dataset', required=True, type=str, help='Dataset, either Tourism or M4')
    parser.add_argument('--frequency', required=True, type=str, help='Dataset time frequency')
    parser.add_argument('--n_random_seeds', required=True, type=int, help='Number of random seeds to test')
    parser.add_argument('--n_lr', required=True, type=int, help='Number of learning rates to test')
    parser.add_argument('--n_iterations', required=True, type=int, help='Number of iterations')
    parser.add_argument('--es_patience', required=True, type=int, help='Early stopping patience')
    
    return parser.parse_args()

In [ ]:
#export
if __name__ == "__main__":
    
#     args = pd.Series({'dataset': 'Tourism',
#                       'frequency': 'Yearly',
#                       'n_random_seeds': 1,
#                       'n_lr': 1,
#                       'n_iterations': 15,
#                       'es_patience': 10})
    args = parse_args()
    main(args)

In [ ]:
# python -m nixtlats.models.nbeats.tuning --dataset M4 --frequency Yearly --n_random_seeds 1 --n_lr 1 --n_iterations 100 --es_patience 10